In [16]:
import pandas as pd



In [17]:
from collections import OrderedDict

In [18]:
import pytz
data = OrderedDict()


In [19]:
data['bank_nifty']=pd.read_csv('bank_nifty.csv',index_col=0,parse_dates=[['Date','Time']])

In [20]:
data['bank_nifty'] = data['bank_nifty'].drop(['Ticker'],axis=1)
data['bank_nifty'].head()

,Open,High,Low,Close,Volume
Date_Time,,,,,
2012-05-30 09:15:00,9599.9004,9599.9004,9561.7998,9566.0000,21375
2012-05-30 09:16:00,9565.0996,9565.0996,9555.1504,9564.4004,15475
2012-05-30 09:17:00,9564.3496,9570.0000,9563.6504,9567.0000,13850
2012-05-30 09:18:00,9568.0000,9569.0000,9561.6504,9563.2500,11250
2012-05-30 09:19:00,9566.5498,9568.0000,9556.0000,9558.3496,16050


In [22]:
data['bank_nifty'] = data['bank_nifty'][['Open','High','Low','Close','Volume ']]

In [23]:
data['bank_nifty'].head()

,Open,High,Low,Close,Volume
Date_Time,,,,,
2012-05-30 09:15:00,9599.9004,9599.9004,9561.7998,9566.0000,21375
2012-05-30 09:16:00,9565.0996,9565.0996,9555.1504,9564.4004,15475
2012-05-30 09:17:00,9564.3496,9570.0000,9563.6504,9567.0000,13850
2012-05-30 09:18:00,9568.0000,9569.0000,9561.6504,9563.2500,11250
2012-05-30 09:19:00,9566.5498,9568.0000,9556.0000,9558.3496,16050


In [24]:
panel = pd.Panel(data)
panel.minor_axis = ["open","high","low","close","volume "]
panel.major_axis = panel.major_axis.tz_localize(pytz.utc)
print(panel)

<class 'pandas.core.panel.Panel'>
Dimensions: 1 (items) x 412117 (major_axis) x 5 (minor_axis)
Items axis: bank_nifty to bank_nifty
Major_axis axis: 2012-05-30 09:15:00+00:00 to 2016-12-09 15:29:00+00:00
Minor_axis axis: open to volume 


In [25]:
import zipline

In [26]:
from zipline.api import order, record, symbol, set_benchmark

In [27]:
import matplotlib.pyplot as plt
from datetime import datetime

In [32]:
def initialize(context):
    context.security = symbol('bank_nifty')


In [34]:
def handle_data(context, data):
    MA1 = data[context.security].mavg(50)
    MA2 = data[context.security].mavg(100)
    date = str(data[context.security].datetime)[:18]
    current_price = data[context.security].price
    current_positions = context.portfolio.positions[symbol('bank_nifty')].amount
    cash = context.portfolio.cash
    value = context.portfolio.portfolio_value
    current_pnl = context.portfolio.pnl
    if(MA1 > MA2) and current_positions == 0:
        num_shares = int(cash/current_price)
        order(context.security,num_shares)
        record(date=date,MA1=MA1,MA2=MA2,price=current_price,status="buy",shares=num_shares,PnL=current_pnl,cash=cash,value=value)
    elif(MA1 < MA2) and current_positions != 0:
        order_target(context.security,0)
        record(date=date,MA1=MA1,MA2=MA2,price=current_price,status='sell',shares='--',PnL=current_pnl,cash=cash,value=value)
    else:
        record(date=date,MA1=MA1,MA2=MA2,price=current_price,status='--',shares='--',PnL=current_pnl,cash=cash,value=value)
            

In [35]:
from zipline import TradingAlgorithm as ta


In [37]:
algo = ta(initialize=initialize,handle_data=handle_data)
perf_manual = algo.run(data)


SymbolNotFound: Symbol 'BANK_NIFTY' was not found.

In [38]:
from zipline.api import(    symbol,
                            get_datetime,
                            order_target_percent,
                            schedule_function,
                            date_rules,
                            time_rules,
                            attach_pipeline,
                            pipeline_output,
                            set_commission,
                            set_slippage,
                            get_open_orders,
                            cancel_order
                       )

In [39]:
from zipline.errors import HistoryWindowStartsBeforeData

In [40]:
def initialize(context):

    context.security_list = [symbol('VEDL'), symbol('MARUTI')]
    schedule_function(strategy, 
                      date_rules.every_day(),
                      time_rules.market_open(hours=2, minutes=30))

In [43]:
def strategy(context, data):
    
    try:
        
        security_data = data.history(assets=context.security_list,
                                 fields='price', 
                                 bar_count=20,
                                 frequency='1d')
    except Exception as e:
        
        if e is HistoryWindowStartsBeforeData:            
            return            
        else:
            raise e

   
    sma = pd.DataFrame()
    sma['5'] = security_data[-5:].mean()
    sma['20'] = security_data.mean()
    
    
     
    for security, orders in get_open_orders().iteritems():       
        for oo in orders:  
             message = 'Canceling order for {amount} shares in {stock}'  
             message = message.format(amount=oo.amount, stock=security.symbol)  
             print(message)  
             cancel_order(oo)       

    # Place orders
    for security in context.security_list:
        if sma.loc[security, '5'] > sma.loc[security, '20']:
            
            order_target_percent(security, 0.50)
            
        else:
            
            order_target_percent(security, -0.50)

In [44]:
from zipline import TradingAlgorithm as ta

In [45]:
algo = ta(initialize=initialize,handle_data=strategy)
perf_manual = algo.run(data)

SymbolNotFound: Symbol 'VEDL' was not found.